In [92]:
import requests 
from bs4 import BeautifulSoup   
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
import re


In [30]:
RANDOM_SEED = 42

In [489]:
test_cars=pd.read_csv('test_cars.csv')
test_cars=test_cars.drop(['car_url', 'complectation_dict',  'image', 'model_info',
                          'parsing_unixtime', 'priceCurrency', 'sell_id', 'super_gen', 'vendor', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38'], axis=1)

test_cars['sample']=1
test_cars



,bodyType,brand,color,description,engineDisplacement,enginePower,equipment_dict,fuelType,mileage,modelDate,...,vehicleConfiguration,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,sample
0,лифтбек,SKODA,синий,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,74000,2013,...,LIFTBACK ROBOT 1.2,роботизированная,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1
1,лифтбек,SKODA,чёрный,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,60563,2017,...,LIFTBACK MECHANICAL 1.6,механическая,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1
2,лифтбек,SKODA,серый,"Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,88000,2013,...,LIFTBACK ROBOT 1.8,роботизированная,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1
3,лифтбек,SKODA,коричневый,КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,95000,2013,...,LIFTBACK AUTOMATIC 1.6,автоматическая,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1
4,лифтбек,SKODA,белый,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,58536,2008,...,LIFTBACK AUTOMATIC 1.8,автоматическая,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34681,седан,BMW,коричневый,Продается BMW 316i в отличном состоянии. Прода...,1.6 LTR,136 N12,"{""cruise-control"":true,""asr"":true,""tinted-glas...",бензин,115000,2011,...,SEDAN AUTOMATIC 1.6,автоматическая,3 или более,2 года и 1 месяц,Оригинал,задний,Левый,Не требует ремонта,Растаможен,1
34682,седан,BMW,чёрный,Продаётся отличный автомобиль.,2.0 LTR,190 N12,NaN,дизель,98000,2016,...,SEDAN AUTOMATIC 2.0,автоматическая,1 владелец,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1
34683,седан,BMW,серый,Хорошее состояние,2.5 LTR,170 N12,NaN,бензин,360000,1995,...,SEDAN AUTOMATIC 2.5,автоматическая,3 или более,NaN,Дубликат,задний,Левый,Не требует ремонта,Растаможен,1
34684,внедорожник 5 дв.,BMW,коричневый,Автомобиль в идеальном состоянии . Куплен новы...,2.0 LTR,184 N12,"{""engine-proof"":true,""esp"":true,""start-stop-fu...",дизель,90500,2012,...,ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,1


In [490]:
data_par = pd.read_csv('all_auto_ru_07_12_2020-200.csv')
data_par=data_par.drop([ 'start_date', 'hidden'], axis=1)
data_par['sample']=0


In [423]:
data = test_cars.append(data_par, sort=False).reset_index(drop=True)


In [420]:
data

,bodyType,brand,color,description,engineDisplacement,enginePower,fuelType,mileage,modelDate,model_name,...,vehicleConfiguration,vehicleTransmission,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
0,лифтбек,SKODA,синий,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,бензин,74000,2013,OCTAVIA,...,LIFTBACK ROBOT 1.2,роботизированная,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,NaN
1,лифтбек,SKODA,чёрный,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,бензин,60563,2017,OCTAVIA,...,LIFTBACK MECHANICAL 1.6,механическая,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,NaN
2,лифтбек,SKODA,серый,"Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,бензин,88000,2013,SUPERB,...,LIFTBACK ROBOT 1.8,роботизированная,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,NaN
3,лифтбек,SKODA,коричневый,КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,бензин,95000,2013,OCTAVIA,...,LIFTBACK AUTOMATIC 1.6,автоматическая,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,NaN
4,лифтбек,SKODA,белый,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,бензин,58536,2008,OCTAVIA,...,LIFTBACK AUTOMATIC 1.8,автоматическая,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38561,Универсал 5 дв. Crossover 7,SUBARU,97948F,Вуаля и вот в продаже семиместный полноприводн...,2.5,173,бензин,128000,2008,EXIGA,...,WAGON_5_DOORS VARIATOR 2.5,VARIATOR,2,NaN,ORIGINAL,полный,RIGHT,NaN,True,1345000.0
38562,Хэтчбек 3 дв.,SUBARU,EE1D19,"Продаю безналогового, малолитражного чистокров...",0.7,54,бензин,158000,2005,R1,...,HATCHBACK_3_DOORS VARIATOR 0.7,VARIATOR,3,"{'year': 2018, 'month': 6}",ORIGINAL,передний,RIGHT,NaN,True,208000.0
38563,Минивэн,SUBARU,0000CC,Автомобиль 8 месяцев в России. Авто второе в с...,0.7,52,бензин,100000,2013,STELLA,...,MINIVAN VARIATOR 0.7,VARIATOR,1,"{'year': 2019, 'month': 9}",ORIGINAL,передний,RIGHT,NaN,True,380000.0
38564,Седан,MAYBACH,CACECB,Автомобиль в идельном состоянии. Модель M57 с ...,5.5,550,бензин,114000,2002,57,...,SEDAN AUTOMATIC 5.5,AUTOMATIC,3,"{'year': 2018, 'month': 5}",ORIGINAL,задний,LEFT,NaN,True,2500000.0


In [93]:
test_cars=pd.read_csv('test_cars.csv')
test_cars=test_cars.drop(['car_url', 'complectation_dict', 'image', 'model_info', 
                          'parsing_unixtime', 'priceCurrency', 'sell_id', 'super_gen', 'vendor', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38'], axis=1)
data_par = pd.read_csv('all_auto_ru_07_12_2020-200.csv')

data_par=data_par.drop([ 'start_date', 'hidden'], axis=1)
test_cars['sample']=1
data_par['sample']=0

data = test_cars.append(data_par, sort=False).reset_index(drop=True)
data.columns =  ['bodyType', 'brand', 'color', 'description', 'engineDisplacement',
       'enginePower', 'equipment_dict', 'fuelType', 'mileage', 'modelDate',
       'model_name', 'name', 'numberOfDoors', 'productionDate',
       'vehicleConfiguration', 'vehicleTransmission', 'owners', 'Possession', 'PTS', 'drive_unit', 'rudder',
       'state', 'choky', 'sample', 'equipment_dict2', 'price']

In [12]:
data.head()

,bodyType,brand,color,description,engineDisplacement,enginePower,equipment_dict,fuelType,mileage,modelDate,...,owners,Possession,PTS,drive_unit,rudder,state,choky,sample,equipment_dict2,price
0,лифтбек,SKODA,синий,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,74000,2013,...,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1,NaN,NaN
1,лифтбек,SKODA,чёрный,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,60563,2017,...,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1,NaN,NaN
2,лифтбек,SKODA,серый,"Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,88000,2013,...,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1,NaN,NaN
3,лифтбек,SKODA,коричневый,КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,95000,2013,...,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1,NaN,NaN
4,лифтбек,SKODA,белый,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,58536,2008,...,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,1,NaN,NaN


Парсинг

Адрес страницы, которую будем парсить

In [34]:
url='https://auto.ru/-/ajax/desktop/listing/'

HEADERS - HTML - элемент, представляет собой вводный контент, обычно группу вводных или навигационных средств. Он может содержать другие элементы-заголовки, а также логотип, форму поиска, имя автора и другие элементы.


Проверим какие столбцы добытого датасета не содержатся в заданом тестовом

In [262]:
def compare(ar1, ar2):
    diff=[]
    for i in ar1:
        if i not in ar2:
            diff.append(i)
    return diff

compare(data_par.columns, test_cars.columns)

['Комплектация', 'price', 'start_date', 'hidden', 'model_name']

In [263]:
compare( test_cars.columns, data_par.columns)

[]

Помечаем добытые и тестовые данные и объединяем их в один датасет

In [458]:
test_cars=pd.read_csv('test_cars.csv')
test_cars=test_cars.drop(['car_url', 'complectation_dict', 'equipment_dict', 'image', 'model_info', 'model_name', 
                          'parsing_unixtime', 'priceCurrency', 'sell_id', 'super_gen', 'vendor', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38'], axis=1)
data_par = pd.read_csv('all_auto_ru_07_12_2020-200.csv')
data_par.shape
#data_par= data_par.drop(['Комплектация', 'start_date', 'hidden', 'model_name'])
test_cars['sample']=1
data_par['sample']=0
data = test_cars.append(data_par, sort=False).reset_index(drop=True)


In [275]:
test_cars=pd.read_csv('test_cars.csv')
test_cars=test_cars.drop(['car_url', 'complectation_dict', 'equipment_dict', 'image', 'model_info', 'model_name',
                          'parsing_unixtime', 'priceCurrency', 'sell_id', 'super_gen', 'vendor', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38'], axis=1)
data_par = pd.read_csv('all_auto_ru_07_12_2020-200.csv')

data_par=data_par.drop(['Комплектация', 'start_date', 'hidden', 'model_name'], axis=1)
test_cars['sample']=1
data_par['sample']=0

data = test_cars.append(data_par, sort=False).reset_index(drop=True)


In [328]:
test_cars=pd.read_csv('test_cars.csv')
test_cars=test_cars.drop(['car_url', 'complectation_dict', 'equipment_dict', 'image', 'model_info', 'model_name',
                          'parsing_unixtime', 'priceCurrency', 'sell_id', 'super_gen', 'vendor', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38'], axis=1)
data_par = pd.read_csv('all_auto_ru_07_12_2020-200.csv')
data_par.shape
test_cars['sample']=1
data_par['sample']=0
data = test_cars.append(data_par, sort=False).reset_index(drop=True)


# Проверим датасет на наличие пропусков:

In [427]:
data.isnull().sum()

bodyType                    2
brand                       0
color                       0
description               146
engineDisplacement          2
enginePower                 2
fuelType                    3
mileage                     0
modelDate                   2
model_name                  0
name                        3
numberOfDoors               2
productionDate              0
vehicleConfiguration        2
vehicleTransmission         2
owners                      0
Possession              25529
PTS                         5
drive_unit                  2
rudder                      0
state                    3880
choky                       0
sample                      0
price                   34686
dtype: int64

Пропуски в некоторых столбцах можно заполнить из общих сведениях о модели машины. Для этого напишим функцию.


In [485]:
'''
test_cars=pd.read_csv('test_cars.csv')
test_cars=test_cars.drop(['car_url', 'complectation_dict', 'equipment_dict', 'image', 'model_info', 
                          'parsing_unixtime', 'priceCurrency', 'sell_id', 'super_gen', 'vendor', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38'], axis=1)
data_par = pd.read_csv('all_auto_ru_07_12_2020-200.csv')

data_par=data_par.drop(['Комплектация', 'start_date', 'hidden'], axis=1)
test_cars['sample']=1
data_par['sample']=0

data = test_cars.append(data_par, sort=False).reset_index(drop=True)
data.columns = ['bodyType', 'brand', 'color', 'description', 'engineDisplacement',
       'enginePower', 'fuelType', 'mileage', 'modelDate', 'model_name', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'owners', 'Possession', 'PTS', 'drive_unit', 'rudder',
       'state', 'choky', 'sample', 'price']


def instna(col):
    for i in range(data.shape[0]):
        #if data.isnull()[i][col]:
        
        if data.iloc[i][col]=="nan": 
           
            br = data.iloc[i]['brand']
            m_n = data.iloc[i]['model_name']
            for j in  range(data.shape[0]):
                if data.iloc[j]['brand']== br and data.iloc[j]['model_name'] == m_n:
                    missing = data.iloc[j][col]
                else:
                    data.drop(i, axis=0)
            data.iloc[i][col] = missing
      

            
instna('bodyType')  
data.isnull().sum()
'''

bodyType                    2
brand                       0
color                       0
description               146
engineDisplacement          2
enginePower                 2
fuelType                    3
mileage                     0
modelDate                   2
model_name                  0
name                        3
numberOfDoors               2
productionDate              0
vehicleConfiguration        2
vehicleTransmission         2
owners                      0
Possession              25529
PTS                         5
drive_unit                  2
rudder                      0
state                    3880
choky                       0
sample                      0
price                   34686
dtype: int64

In [482]:
list(data['bodyType'].astype("str").value_counts().index)


['седан',
 'внедорожник 5 дв.',
 'лифтбек',
 'хэтчбек 5 дв.',
 'универсал 5 дв.',
 'минивэн',
 'Седан',
 'купе',
 'Внедорожник 5 дв.',
 'компактвэн',
 'хэтчбек 3 дв.',
 'Хэтчбек 5 дв.',
 'Универсал 5 дв.',
 'пикап двойная кабина',
 'Купе',
 'Хэтчбек 3 дв.',
 'Минивэн',
 'купе-хардтоп',
 'Компактвэн',
 'внедорожник 3 дв.',
 'родстер',
 'фургон',
 'Кабриолет',
 'кабриолет',
 'Внедорожник 3 дв.',
 'Лифтбек',
 'Родстер',
 'Пикап Двойная кабина',
 'Седан Long',
 'Купе-хардтоп',
 'Минивэн Long',
 'Микровэн',
 'Минивэн Grand',
 'Пикап Полуторная кабина',
 'Внедорожник открытый',
 'седан-хардтоп',
 'Компактвэн Grand',
 'Седан 2 дв.',
 'Пикап Одинарная кабина',
 'Лифтбек Sportback',
 'микровэн',
 'лимузин',
 'седан 2 дв.',
 'Хэтчбек 3 дв. Cooper S',
 'Лифтбек Gran Turismo',
 'Фургон',
 'Хэтчбек 5 дв. Sportback',
 'пикап полуторная кабина',
 'пикап одинарная кабина',
 'Седан Gran Coupe',
 'Седан-хардтоп',
 'Минивэн SWB',
 'Седан Type-S',
 'Пикап Двойная кабина Double Cab',
 'Пикап Двойная кабина

In [461]:
data.isnull().sum()

bodyType                    2
brand                       0
color                       0
description               146
engineDisplacement          2
enginePower                 2
fuelType                    3
mileage                     0
modelDate                   2
name                        3
numberOfDoors               2
productionDate              0
vehicleConfiguration        2
vehicleTransmission         2
Владельцы                   0
Владение                25529
ПТС                         5
Привод                      2
Руль                        0
Состояние                3880
Таможня                     0
sample                      0
Комплектация            34686
price                   34686
start_date              34686
hidden                  38566
model_name              34686
dtype: int64

Некоторые признаки хоть и выглядят как числовые имеют тип стринг. Выполним преобразование.

In [94]:
def splitting(s):
    return str(s).split( ' ')[0]
data["enginePower"]=data["enginePower"].map(lambda s: splitting(s))
data['name']=data['name'].map(lambda s: splitting(s))
data['engineDisplacement'] = data['engineDisplacement'].map(lambda s: splitting(s))
data['owners']=data['owners'].map(lambda s: splitting(s))

In [95]:
def convert_to_int(col):
    def reg_clien(stringa):
        res = re.sub(pattern='[^0-9]', repl='', string=str(stringa))
        if res == '':
            return 0
        else:
            return res    
    data[col] = data[col].fillna('0')
    data[col] = data[col].map(lambda a: reg_clien(a))
    data[col] = data[col].astype(np.float64)
    #data[col].astype(np.int32, inplace=True)
        

#convert_to_int('mileage')
for col in ['mileage', "numberOfDoors", "productionDate", "enginePower", 'name', 'engineDisplacement', 'owners', 'modelDate']:
    convert_to_int(col)

data['modelDate']

0         2013.0
1         2017.0
2         2013.0
3         2013.0
4         2008.0
          ...   
38561    20080.0
38562    20050.0
38563    20130.0
38564    20020.0
38565    20020.0
Name: modelDate, Length: 38566, dtype: float64

In [40]:
data["productionDate"].sort_values(ascending=False)

33674    2020.0
34636    2020.0
24953    2020.0
8581     2020.0
15654    2020.0
          ...  
936         0.0
10806       0.0
28046       0.0
13618       0.0
10065       0.0
Name: productionDate, Length: 38566, dtype: float64

!!!!!!!!!!!!!!!!!!!!!!!!! не забудь сделать хитмап!!! 'name' "engineDisplacement" коррелируют

Dictionary

Превратим длинные строковые значения в колонках 'equipment_dict' в списки.

In [96]:


def eq_to_list(col):
    #data[new_name] = pd.Series()
    #data[col] = data[col].fillna(0)
    for i in range(data.shape[0]):
       
        i_list=str(data.iloc[i][col]).replace(':true', '').replace('{', '').replace('}', '').replace('"', '').split(',')
        data.set_value(i, col, i_list)
       





In [97]:
eq_to_list('equipment_dict')
#eq_to_list('equipment_dict2')

C:\Users\trigu\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys


Значения столбца 'equipment_dict2' немного отличаются от 'equipment_dict', поэтому немного изменим функцию для его преобразования в список.


In [98]:
def eq_to_list2(col):
    #data[new_name] = pd.Series()
    #data[col] = data[col].fillna(0)
    for i in range(data.shape[0]):
        try:
            i_list=str(data.iloc[i][col]).replace('{', '').replace('}', '').split('available_options').pop(1)
            i_list=i_list.replace('{', '').replace('}', '').split("'photos':").pop(0)
            i_list=i_list.replace("'", "").replace(':', '').replace('[', '').replace(']', '').split("'vendor_colors':").pop(0)
            i_list=i_list.split(', ')
            
        except:
            continue
        
        #i_list=str(data.iloc[i][col]).replace(':true', '').replace('{', '').replace('}', '').replace('"', '').split(',')
        data.set_value(i, col, i_list)
eq_to_list2('equipment_dict2')


C:\Users\trigu\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app


In [99]:
data['equipment_dict2']



{'id': '0'}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [103]:
data['equipment_dict2'].value_counts()


{'id': '0'}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

Создадим сприсок из различных уникальных значений комплектации, превратим его в серию и оставим зпначения, которые посторяются чаще 10000 раз. Создадим бинарные фичи из этих значений.

In [101]:
equip=[]
for car in data['equipment_dict']:
    try:
        for eq in car:
            equip.append(eq)
 
    except:
        continue
equipment=pd.Series(equip)
equipment=equipment.value_counts()
equipment=equipment[equipment.values>10000]
equipment

lock                    21145
abs                     20760
electro-mirrors         18788
airbag-driver           17650
computer                17480
electro-window-front    17429
front-seats-heat        17320
airbag-passenger        17196
ptf                     16159
immo                    15935
electro-window-back     15247
airbag-side             14668
aux                     14443
wheel-power             14342
esp                     14146
mirrors-heat            13940
nan                     13872
wheel-leather           13579
multi-wheel             13025
usb                     12943
front-centre-armrest    12842
wheel-configuration1    12563
rain-sensor             12546
light-sensor            12353
bluetooth               11896
cruise-control          11866
light-cleaner           11762
12v-socket              11750
tinted-glass            11658
leather                 10796
alloy-wheel-disks       10440
wheel-configuration2    10198
park-assist-r           10195
isofix    

In [90]:

for col in equipment.index:
    def eq_col(row):
        return int(col in row['equipment_dict'])
    
    data[col]=data.apply(lambda row: eq_col(row), axis=1)
    

     

In [102]:
for col in equipment.index:
    def eq_col(row):
        return int((col in row['equipment_dict']) or (col in row['equipment_dict2']) )
    
    data[col]=data.apply(lambda row: eq_col(row), axis=1)

TypeError: ("argument of type 'float' is not iterable", 'occurred at index 44')

In [91]:
data

,bodyType,brand,color,description,engineDisplacement,enginePower,equipment_dict,fuelType,mileage,modelDate,...,cruise-control,light-cleaner,12v-socket,tinted-glass,leather,alloy-wheel-disks,wheel-configuration2,park-assist-r,isofix,seat-transformation
0,лифтбек,SKODA,синий,"Все автомобили, представленные в продаже, прох...",12.0,105.0,"[engine-proof, tinted-glass, airbag-driver, au...",бензин,74000.0,2013.0,...,0,0,1,1,0,0,1,1,1,1
1,лифтбек,SKODA,чёрный,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,16.0,110.0,"[cruise-control, asr, esp, airbag-driver, isof...",бензин,60563.0,2017.0,...,1,0,0,0,0,1,1,1,1,0
2,лифтбек,SKODA,серый,"Все автомобили, представленные в продаже, прох...",18.0,152.0,"[cruise-control, tinted-glass, esp, adaptive-l...",бензин,88000.0,2013.0,...,1,1,1,1,0,1,1,1,1,1
3,лифтбек,SKODA,коричневый,КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,16.0,110.0,"[cruise-control, roller-blind-for-rear-window,...",бензин,95000.0,2013.0,...,1,1,1,1,0,1,1,1,1,1
4,лифтбек,SKODA,белый,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,18.0,152.0,"[cruise-control, asr, esp, airbag-driver, isof...",бензин,58536.0,2008.0,...,1,0,0,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38561,Универсал 5 дв. Crossover 7,SUBARU,97948F,Вуаля и вот в продаже семиместный полноприводн...,25.0,1730.0,[nan],бензин,128000.0,20080.0,...,0,0,0,0,0,0,0,0,0,0
38562,Хэтчбек 3 дв.,SUBARU,EE1D19,"Продаю безналогового, малолитражного чистокров...",7.0,540.0,[nan],бензин,158000.0,20050.0,...,0,0,0,0,0,0,0,0,0,0
38563,Минивэн,SUBARU,0000CC,Автомобиль 8 месяцев в России. Авто второе в с...,7.0,520.0,[nan],бензин,100000.0,20130.0,...,0,0,0,0,0,0,0,0,0,0
38564,Седан,MAYBACH,CACECB,Автомобиль в идельном состоянии. Модель M57 с ...,55.0,5500.0,[nan],бензин,114000.0,20020.0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
for eq in equipment.index:






for i in range(100): #(data.shape[0]):
    for eq in equipment.index:
        #print(eq)
        try: 
            
            print(data[i]['equipment_dict'])
            if eq in data[i]['equipment_dict'] :
                
                data.set_value(i, eq, 1)
          
        except:
            continue
        
#data

    

In [68]:
data['equipment_dict']

0        [engine-proof, tinted-glass, airbag-driver, au...
1        [cruise-control, asr, esp, airbag-driver, isof...
2        [cruise-control, tinted-glass, esp, adaptive-l...
3        [cruise-control, roller-blind-for-rear-window,...
4        [cruise-control, asr, esp, airbag-driver, isof...
                               ...                        
38561                                                [nan]
38562                                                [nan]
38563                                                [nan]
38564                                                [nan]
38565                                                [nan]
Name: equipment_dict, Length: 38566, dtype: object

In [56]:
data["cruise-control"].value_counts()

0    38566
Name: cruise-control, dtype: int64

In [4]:
equipment=[]

for i in range(3):
    i_list=data.iloc[i]['equipment_dict'].replace(':true', '').replace('{', '').replace('}', '').replace('"', '').split(',')
    
    for j in i_list:
         
        if j  not in equipment:
            #print(type(j))
            equipment.append(j)
equipment





['engine-proof',
 'tinted-glass',
 'airbag-driver',
 'aux',
 'isofix',
 'electro-window-front',
 'ashtray-and-cigarette-lighter',
 'airbag-passenger',
 'computer',
 'high-beam-assist',
 'seat-transformation',
 'isofix-front',
 'wheel-power',
 'alarm',
 'lock',
 'door-sill-panel',
 'fabric-seats',
 'electro-mirrors',
 'airbag-rear-side',
 'electro-window-back',
 'steel-wheels',
 'ptf',
 '16-inch-wheels',
 'rain-sensor',
 'airbag-side',
 'audiosystem-cd',
 'dark-interior',
 'cooling-box',
 'condition',
 'abs',
 'power-child-locks-rear-doors',
 'front-seats-heat',
 'wheel-configuration2',
 'wheel-configuration1',
 '12v-socket',
 'park-assist-r',
 'cruise-control',
 'asr',
 'esp',
 'usb',
 'light-sensor',
 'drl',
 'alloy-wheel-disks',
 'tyre-pressure',
 'mirrors-heat',
 'driver-seat-updown',
 'wheel-heat',
 'bluetooth',
 'immo',
 'windscreen-heat',
 'climate-control-2',
 'adaptive-light',
 'multi-wheel',
 'xenon',
 'heated-wash-system',
 'front-centre-armrest',
 'body-mouldings',
 'combo-i

In [394]:
def date_devide(d):
    if d>19000 and d<=20200:
        d=d/10
    else:
        d
    return d
data['modelDate']=data['modelDate'].apply(date_devide)
data['modelDate']

0        2013.0
1        2017.0
2        2013.0
3        2013.0
4        2008.0
          ...  
38561    2008.0
38562    2005.0
38563    2013.0
38564    2002.0
38565    2002.0
Name: modelDate, Length: 38566, dtype: float64

In [374]:
data['modelDate'].sort_values(ascending=True)

38163    1951.0
38211    1951.0
34693    1953.0
34730    1953.0
34729    1953.0
          ...  
38090       NaN
38095       NaN
38099       NaN
38223       NaN
38322       NaN
Name: modelDate, Length: 38566, dtype: float64

In [327]:
def splitting(s):
    return str(s).split( ' ')[0]
data["enginePower"]=data["enginePower"].map(lambda s: splitting(s))


0          105
1          110
2          152
3          110
4          152
         ...  
38561    173.0
38562     54.0
38563     52.0
38564    550.0
38565    612.0
Name: enginePower, Length: 38566, dtype: object

In [ ]:
for col in data[['mileage', "numberOfDoors", "productionDate" ]]:
    convert_to_int(col)

In [239]:
test_cars=pd.read_csv('test_cars.csv')
test_cars=test_cars.drop(['car_url', 'complectation_dict', 'equipment_dict', 'image', 'model_info', 'model_name',
                          'parsing_unixtime', 'priceCurrency', 'sell_id', 'super_gen', 'vendor', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37', 'Unnamed: 38'], axis=1)
data_par = pd.read_csv('all_auto_ru_07_12_2020-200.csv')
data_par.shape
test_cars['sample']=1
data_par['sample']=0
data = test_cars.append(data_par, sort=False).reset_index(drop=True)

In [254]:
for col in data[['mileage', "numberOfDoors", "productionDate" ]]:
    convert_to_int(col)
    

In [201]:
 data[['mileage', "numberOfDoors", "productionDate" ]]

,mileage,numberOfDoors,productionDate
0,74000,5,2014
1,60563,5,2017
2,88000,5,2014
3,95000,5,2014
4,58536,5,2012
...,...,...,...
38587,128296,50,2015
38588,17700,50,2019
38589,130000,50,2013
38590,154000,50,2006


In [224]:
type(data["productionDate"][3])

str

In [210]:
 data["productionDate"].value_counts()

2012    2968
2008    2824
2011    2750
2013    2637
2014    2332
        ... 
1932       1
1954       1
1927       1
1942       1
1962       1
Name: productionDate, Length: 84, dtype: int64

In [162]:
i=  '344fg'
i=re.sub(pattern='[^0-9]', repl='', string=i)
i=int(i)
i

344

In [166]:
data['mileage'][3]

'95000'

In [167]:
type(data['mileage'][3])

str

In [129]:
for col in data[['mileage', "numberOfDoors", "productionDate" ]]:
    data[col]=(re.sub(repl='[^0-9]'), '', data[col])
    data[col].astype("float")

TypeError: sub() missing 2 required positional arguments: 'pattern' and 'string'

# Обработаем категориальные признаки

К большинству категориальных признаков можно применить  LabelEncoder. Для этого объедентим их заголовки в список и создадим функцию для быстрого преминения энкодера.

In [90]:
cat_encod=['bodyType', 'brand', 'color', 'fuelType', 'priceCurrency', 'vehicleConfiguration','vehicleTransmission', 'vendor']

Применим к выбранным категориальнвм столбцам LabelEncoder, чтоб преобразовать его значения в числовые

In [91]:
le =preprocessing.LabelEncoder()

for col in data[cat_encod]:
    le.fit(list(data[col].values))
    data[col]=le.transform(list(data[col].values))
    

In [73]:
le =preprocessing.LabelEncoder()
le.fit(list(data['bodyType'].values))
data['bodyType']=le.transform(list(data['bodyType'].values))


In [84]:
data

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,sample,Комплектация,price,start_date,hidden,model
0,59,21,https://auto.ru/cars/used/sale/skoda/octavia/1...,28,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
1,59,21,https://auto.ru/cars/used/sale/skoda/octavia/1...,30,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
2,59,21,https://auto.ru/cars/used/sale/skoda/superb/11...,27,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
3,59,21,https://auto.ru/cars/used/sale/skoda/octavia/1...,21,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
4,59,21,https://auto.ru/cars/used/sale/skoda/octavia/1...,16,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35226,35,20,NaN,9,NaN,"в хорошем состоянии заменен грм полностью,бенз...",1.8,125,NaN,бензин,...,NaN,NaN,NaN,NaN,0,{'id': '0'},250000.0,2020-11-17T12:59:56Z,NaN,TOLEDO
35227,47,20,NaN,11,NaN,К сожалению обстоятельства заставляют расстать...,2.0,150,NaN,бензин,...,NaN,NaN,NaN,NaN,0,{'id': '0'},375000.0,2020-11-13T09:40:51Z,NaN,TOLEDO
35228,20,20,NaN,1,NaN,NaN,1.6,75,NaN,бензин,...,NaN,NaN,NaN,NaN,0,{'id': '0'},125000.0,2020-11-27T06:44:15Z,NaN,TOLEDO
35229,32,25,NaN,2,NaN,"Хозяин авто умер,по машине ничего не знаю,заво...",2.4,126,NaN,бензин,...,NaN,NaN,NaN,NaN,0,{'id': '0'},214000.0,2020-09-08T14:52:01Z,NaN,GRAND_TIGER


In [110]:
)

axample atring
